In [133]:
import pandas as pd
import numpy as np
#from taylor_eda import fraud_column
from sklearn.svm import LinearSVC
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import matplotlib.pylab as plt
#import pickle

def fraud_column(df):
    print('In fraud_column')
    '''
    Add column called fraud based on acct_type column.
    Input: initial dataframe without labeled fraud
    Returns: dataframe with label of fraud (1) or not fraud (0)
    '''

    df['fraud'] = df.apply(label_fraud, axis=1)
    df_final = df.drop('acct_type', axis=1)
    return df_final

def label_fraud(row):
    '''
    Label one row of fraud dataframe at a time. This is easy to make more generalized by adding elifs to the function.
    Input: Unlabeled row in the fraud dataframe
    Returns: Labeled row. Fraud is labeled if the account type is 'fraudster_event', 'fraudster', 'fraudster_att'
    '''

    # fraud_labels are row values in the account type column that will result in the row being labeled fraud (1)
    fraud_labels = ['fraudster_event', 'fraudster', 'fraudster_att']

    if row['acct_type'] in fraud_labels:
        val = 1
        #print(1)
    else:
        val = 0
        print(val)
    return val



def preprocess(df_fraud):
    '''
    Get the data ready to build a model. Take a pandas dataframe and creates a train test split and makes numpy arrays that are ready to be classified.
    Input: pandas dataframe with labels in 'fraud' column
    Return: X_train, X_test, y_train, y_test as numpy arrays
    '''

    y = df_fraud.pop('fraud').values
    # Use df_fraud.values once the data is cleaned and all numeric
    X = df_fraud.values
    # Use just a few numeric, full columns to get the model working at the start
    # X = df_fraud[['user_age', 'user_type', 'channels']].values

    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

    return X_train, X_test, y_train, y_test

def gridsearch(X_train, y_train, params, model):
    '''
    Look for hyperparameters in a SVC model
    Input:
        (1), (2) training data already broken up into X_train and y_train numpy arrays
        (3) a grid of paramters to gridsearch through to make a model
        (4) the sklearn model that will be used in the gridsearch
    Return: The best model that the gridsearch came up with. This model will be fit to the data.
    '''

    #make a model
    mdl = model

    do_search = lambda x: GridSearchCV(estimator=x,
                                       param_grid=params,
                                       cv=3,
                                       scoring='recall',
                                       verbose=3,
                                       return_train_score = True,
                                       n_jobs=-1) \
                                       .fit(X_train, y_train)

    # Gridsearch a SVC model based on the parameter list
    grid_model = do_search(mdl)

    # Get the fit SVC model with the best parameters from the gridsearch
    model_fit = grid_model.best_estimator_
    return model_fit, grid_model

def validation_score(model, X_test, y_test):
    '''
    Uses the model that was created in the gridsearch function to predict categories for the X_test data and see how well the model did.
    f1 score being used to evaluate the preformance of the model
    Input:
        (1) model that preformed the best in the gridsearch
        (2) X data that the model hasn't seen at all yet to validate the model
        (3) True y labels that will be used to score the model
    Return: f1, precision and recall scores based on the previously unseen validataion data
    '''

    y_pred = model.predict(X_test)

    valid_f1 = f1_score(y_test, y_pred)
    valid_prec = precision_score(y_test, y_pred)
    valid_rec = recall_score(y_test, y_pred)

    print ('   validation set precision score: {0:.2f}.'.format(valid_prec))
    # print '   this is the ability of the classifier not to label a legal sample as fraudulent.'
    print ('   {0:.2f}% of the fraud that our model pointed out was actually fraud.'.format(100*valid_prec))
    print ('')
    print ('   validation set recall score: {0:.3f}.'.format(valid_rec))
    # print '   this is the ability of the classifier not to label a fraudulent sample as legal.'
    print ('   {0:.2f}% of the fraud coming in was actually called out by out model.'.format(100*valid_rec))
    print ('')
    print ('   validation set f1 score: {0:.3f}.'.format(valid_f1))
    print ('   this is a metric that combines the precision and recall scores.')

    return valid_f1, valid_prec, valid_rec

if __name__ == '__main__':
    np.random.seed(60)

    #filename = 'clean_df_5.csv'
    #df = pd.read_csv(filename)
    dataset_LicenseNumber
    # df = df.drop(['old_index','new_index'],axis = 1)
    #df_fraud = fraud_column(df)
    df_fraud = fraud_column(dataset_LicenseNumber)
    print(df_fraud)
    # df_fraud.sort_index(axis = 1, inplace = True)
    # svc will be trained and tested through cross validated
    #X_train, X_test, y_train, y_test = preprocess(df)
    
    #df_fraud=df_fraud.drop('drivernid', axis=1)
    X_train, X_test, y_train, y_test = preprocess(df_fraud)
    
    #dictionary of hyperparameters to test
    #ADD MORE TO SEARCH THROUGH!!
    # params = {'C':np.linspace(.001, 3, 3)}
    '''
    RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)
    '''
    # note that we have 644 features
    # looping over features to plot affects
    ns = [0.25] # max features
    ns = [5, 10, 20, 40, 80, 160] # n estimators could do 320
    ns = [1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 20]# min samples leaf
    ns = [1,2,3,4] # to loop over just one
    trains = []
    tests = []
    #2nd one
    # Create the parameter grid based on the results of random search 
    param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [1.0],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
    }
    
    # Create a based model
    #rf = RandomForestRegressor()
    model = RandomForestClassifier() #AdaBoostClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
    
    grid_result=grid_search.fit(X_train,y_train)
    #model = RandomForestClassifier() #AdaBoostClassifier()
    #mdl_fit, grid = gridsearch(X_train, y_train, params, model)
    #summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
         print("%f (%f) with: %r" % (mean, stdev, param))
    #2nd one end
        #mdl_fit, grid = gridsearch(X_train, y_train, params, model)
        #print ('----test')
        #print ('')
        #f1, prec, rec = validation_score(mdl_fit, X_test, y_test)
        #tests.append([f1,prec,rec])
        #print ('')
        #print ('-'*20)
        #print ('')
        #print ('----train')
        #print ('')
        #f1, prec, rec = validation_score(mdl_fit, X_train, y_train)
        #trains.append([f1, prec, rec])
    
    for n in ns:
        #print(n)
        params = {
        'bootstrap': [True],
        'max_depth': [80, 90, 100, 110],
        'max_features': [1.0],
        'min_samples_leaf': [3, 4, 5],
        'min_samples_split': [8, 10, 12],
        'n_estimators': [100, 200, 300, 1000]
        }
        
        #params = {'n_estimators': [40], #higher
                  #'min_samples_leaf': [8], #lower
                  #'max_features': [1.0], #higher
                  #'min_samples_split': [3], #higher
                  #'oob_score': [False], #True
                  #'n_jobs': [10]
                  #}

        model = RandomForestClassifier() #AdaBoostClassifier()
            
        mdl_fit, grid = gridsearch(X_train, y_train, params, model)
        print ('----test')
        print ('')
        f1, prec, rec = validation_score(mdl_fit, X_test, y_test)
        tests.append([f1,prec,rec])
        print ('')
        print ('-'*20)
        print ('')
        print ('----train')
        print ('')
        f1, prec, rec = validation_score(mdl_fit, X_train, y_train)
        trains.append([f1, prec, rec])
        print(n)
# -------- plotting the variances when changing the
    # plt.style.use('ggplot')
    # f1, prec, rec = zip(*trains)
    # plt.plot(ns,f1,linestyle = '-.',color = 'b',label = 'Train F1')
    # plt.plot(ns,prec,linestyle = '-.',color = 'r',label = 'Train Precision')
    # plt.plot(ns,rec,linestyle = '-.',color = 'g',label = 'Train Recall')
    # f1, prec, rec = zip(*tests)
    # plt.plot(ns,f1,color = 'b',label = 'Test F1')
    # plt.plot(ns,prec,color = 'r',label = 'Test Precision')
    # plt.plot(ns,rec,color = 'g',label = 'Test Recall')
    # plt.xlabel('min_samples_leaf',fontsize = 20)
    # plt.ylabel('Score',fontsize = 20)
    # # plt.legend()
    # # plt.xscale('log')
    # plt.title('6 points, n_estimators = 40, max_features = 80', fontsize = 20)
    # plt.show ()


    # ---------- top features
    top = 30

    feats = mdl_fit.feature_importances_
    i = np.argsort(feats)[::-1]
    cols = np.array(df_fraud.columns.tolist())
    top_n = cols[i][:top]
    top_feats = feats[i][:top]
    tops = zip(top_n,top_feats)
    print (tops)

    '''
----test
   validation set precision score: 0.96.
   95.66% of the fraud that our model pointed out was actually fraud.
   validation set recall score: 0.853.
   85.31% of the fraud coming in was actually called out by out model.
   validation set f1 score: 0.902.
   this is a metric that combines the precision and recall scores.
--------------------
----train
   validation set precision score: 0.97.
   96.96% of the fraud that our model pointed out was actually fraud.
   validation set recall score: 0.917.
   91.71% of the fraud coming in was actually called out by out model.
   validation set f1 score: 0.943.
   this is a metric that combines the precision and recall scores.
# -----------------------
   top features
 [('quantity_sold', 0.30129828800520336),
 ('sale_duration2', 0.1070573949089052),
 ('pay_other', 0.084884062414186098),
 ('sale_duration', 0.052883591953294541),
 ('user_age', 0.050665812219055306),
 ('gts', 0.047329122753276734),
 ('num_payouts', 0.035383200134365946),
 ('user_type', 0.023739330213435352),
 ('cost', 0.01693296238279306),
 ('has_venue_longitude', 0.010934629470331943),
 ('delivery_method', 0.0087529873849410151),
 ('has_org_name', 0.0079926506335234234),
 ('body_length', 0.007656685738912203),
 ('has_event_published', 0.0057132136445213555),
 ('nlp_293', 0.0050127166065724611),
 ('pay_check', 0.0046048814461881219),
 ('name_length', 0.0044913919930811936),
 ('quantity_total', 0.0038346638574849627),
 ('has_venue_latitude', 0.0037130341905840269),
 ('nlp_178', 0.0034906396282950746),
 ('has_description', 0.0033163817209423941),
 ('lat', 0.0032098326895639772),
 ('nlp_44', 0.0031401549420813891),
 ('has_payee_name', 0.0029241624672553769),
 ('nlp_122', 0.0028707325124463145),
 ('nlp_161', 0.0028513609639768183),
 ('nlp_117', 0.0026447287284730109),
 ('has_venue_address', 0.0025187624785950329),
 ('nlp_52', 0.0025090011220533903),
 ('has_venue_name', 0.0025082686738964978)]
    '''
    # pickle.dump( mdl_fit, open( "random_forest.pkl", "wb"))

In fraud_column
      is_deleted  is_blocked  user_id  phone_no  date_registered  \
0              0           0   183299       151              880   
1              0           0   199528       150             1252   
2              0           1    72249       500              929   
3              0           0   197850       819             1203   
4              0           0   105568       338              471   
5              0           0   183361       449              881   
6              0           0    54395       751              623   
7              0           0     5705       513              405   
8              0           0   186905       211              946   
9              0           0    25850       168              710   
10             0           0    30693       366              810   
11             0           0    77088       196             1068   
12             1           1    74943      1196             1021   
13             0           0    

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.5min finished


Best: 1.000000 using {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 200}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 1000}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
1.000000 (0.000000) with: {'bootstrap': True, 'max_depth': 80, 'max_features': 1.0, 'min_sa

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.5min finished


----test

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.

--------------------

----train

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.
1
Fitting 3 folds for each of 144 candidates, totalling 432 fits


...

----test

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.

--------------------

----train

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.
2
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.5min finished


----test

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.

--------------------

----train

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.
3
Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  1.6min finished


----test

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.

--------------------

----train

   validation set precision score: 1.00.
   100.00% of the fraud that our model pointed out was actually fraud.

   validation set recall score: 1.000.
   100.00% of the fraud coming in was actually called out by out model.

   validation set f1 score: 1.000.
   this is a metric that combines the precision and recall scores.
4


In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import read_csv

In [42]:
dataset_NID=read_csv("C:/Users/HP/Desktop/Fraud Documents/NID.csv")
dataset_LicenseNumber=read_csv("C:/Users/HP/Desktop/Fraud Documents/LicenseNumber.csv")
dataset_Chassis=read_csv("C:/Users/HP/Desktop/Fraud Documents/Chassis.csv")
dataset_NumberPlate=read_csv("C:/Users/HP/Desktop/Fraud Documents/NumberPlate.csv")

In [43]:
dataset_NID

,is_deleted,is_blocked,user_id,phone_no,date_registered,total_rides_as_driver,unique_device_id,device_name,os_version,app_versioncode,...,profession,present_district,sales_person,crm_name,per_block,username,driverlicense,drivernid,vehiclechassis,vehicleplate
0,0,0,137423,8801625235288,30/06/2018 11:25:40 AM,103,867397030327680,XiaomiRedmi Note 4,7.0,117,...,NaN,Dhaka,Fixed Rider,Md Sajib Hossain,"H-KA-11/11, Norda, Vatara, Gulshan-1212, Dhaka",RIADMAHMUDROMAN,4511590610,1026225936,MD2A76AYOHWH89988,DHAKAMETROHA323046
1,0,1,122150,8801878940611,23/06/2018 4:27:37 AM,5,NaN,XiaomiRedmi Note 4,7.0,116,...,--,Dhaka,Sales Force-UTTARA004,Md Shemul Ahammed,"H-KA-11/11, Norda, Vatara, Gulshan, Dhaka",RIADMAHMUDROMAN,4511590610.,1026225936,MD625BF46N1N21822,DHAKAMETROLA322735
2,0,0,197560,8801680170769,29/07/2018 5:26:47 AM,0,NaN,NaN,NaN,NaN,...,NaN,Dhaka,Sales Force-MHK280,Sharmin Akhter Rima,"375 East Nakhalpara, Tejgaon Dhaka,1215",KAZIMAHFUJURURRAHMANASHIK,211373500,1489196343,MD2A11CY5HWB93342,DHAKAMETROLA342944
3,0,0,204749,880168418447,02/08/2018 4:48:36 AM,0,NaN,NaN,NaN,NaN,...,NaN,Dhaka,Sales Force-BNS1214,Sharmin Akhter Rima,"375 East Nakhalapara, Tejgao Shilpo Alaka-1225...",KAZIMAHFUZURRAHMANASHIK,211373500,1489196343,MD2A11CY5HW893342,DHAKAMETROLA342944
4,0,0,69668,8801999955899,23/05/2018 5:42:55 AM,3,353505071576327,samsungSM-J500H,6.0.1,109,...,NaN,Dhaka,Sales Force-JRB420,Md Habibur Rahman,"H- N Colony, 14 no. Outfall, Wari TSO-1213, Ja...",MDSAWPONMIAH,311491269,1901456275,LBBPEGTAXHB853562,DHAKAMETROHA324416
5,0,0,183228,8801747172215,22/07/2018 6:15:32 AM,0,357775214478642,LAVAPixel V2+,5.1,117,...,NaN,Dhaka,Sales Force-JRB666,Dibakar Chandra Das,"N. Colony, 14 No Outful, Brahmon Chiron, Wari ...",MDSAWPONMIAH,311491269,1901456275,MD2DHDHZZUCC05075,DHAKAMETROLA193565
6,0,0,54022,8801612222119,15/05/2018 6:22:17 AM,0,NaN,NaN,NaN,NaN,...,NaN,Dhaka,Activation Campaign-Expoexpert786,Sarah Jabin,"House-123, Road-Hazaribag, PO- Zigatola T.S.O-...",MDAMIRPARVEZ,211460630,1924285313,ME4KCO93M98116504,DHAKAMETROLA150767
7,0,0,105205,8801792121212,11/06/2018 6:40:44 AM,0,NaN,NaN,NaN,NaN,...,NaN,Dhaka,Activation Campaign-Expoexpert786,Md Sajib Hossain,"H# 123, Hazaribagh, Jigatala TSO-1209, Hazarib...",MDAMIRPARVEZ,2114606300,1924285313,ME4KCO93M98116504.,DHAKAMETROLA150767
8,0,0,211816,8801773762564,06/08/2018 6:43:06 AM,0,NaN,NaN,NaN,NaN,...,NaN,Dhaka,CNG-BP-Enamul Haque,Md Habibur Rahman,"H-15/KA/1, Mirbag, Santinagar, Ramna, Dhaka",MDFORHADHOSSAIN,DK0067045TL0009,1924938390,ACFBKE00222,DHAKAMETROTAW134585
9,0,0,129811,8801923535639,27/06/2018 4:59:12 AM,0,NaN,NaN,NaN,NaN,...,--,Dhaka,CNG-BP- Foyaze Ahmed,Jawaad Rayees,"15/KA/1, Mirbagh, Shantinagar, Ramna, Dhaka",MDFORHADHOSSAIN,PENDING01923,1924938390,FFBJG06964,DHAKAMETROTAW141165


In [45]:
#dataset_LicenseNumber=DataFrame()
dataset_LicenseNumber['acct_type'] = pd.Series('fraudster', index=dataset_LicenseNumber.index)

In [46]:
dataset_LicenseNumber

,is_deleted,is_blocked,user_id,phone_no,date_registered,total_rides_as_driver,unique_device_id,device_name,os_version,app_versioncode,...,present_district,sales_person,crm_name,per_block,username,driverlicense,drivernid,vehiclechassis,vehicleplate,acct_type
0,0,0,183299,+8801679350974,22/07/2018 6:48:10 AM,0,NaN,NaN,NaN,NaN,...,Dhaka,Sales Force-UTT022,Rezwana Hafiz,"85731901701,Adom,Paotanahat-5450,Pirgacha Pall...",MDSHARIFULISLAM,21148990,19938517319000300,MD2A14AZ4CWC85046,RANGPURHA135222,fraudster
1,0,0,199528,+8801679350497,30/07/2018 6:04:27 AM,0,NaN,NaN,NaN,117,...,Dhaka,Sales Force-KB1800,Sharmin Akhter Rima,"Shafiqul Vila,Pawtanahat-545 Pirgacha Palli,Ra...",MDSHARIFULISLAM,21148990,19938517319000300,MD2A14AZ4CWC85046,RANGPURHA135222,fraudster
2,0,1,72249,+8801777283248,24/05/2018 1:31:45 PM,6,NaN,XiaomiRedmi Note 5A,7.1.2,117,...,Dhaka,Sales Force-KB100,Md Sajib Hossain,"North Para , Kundapara , Jumarbari , Saghata ,...",MDSULTANMAHAMUD,21151376,5982420423,DHAKAMETROHA347297,DHAKAMETROHA347297,fraudster
3,0,0,197850,+8801923456326,29/07/2018 7:41:31 AM,0,867708031750581,XiaomiRedmi Note 5A,7.1.2,117,...,Dhaka,Fixed Rider,Md Sajib Hossain,"North Para , Kundapara , Jumarbari-5750, Sagha...",MDSULTANMAHAMUD,21151376,5982420423,NaN,NaN,fraudster
4,0,0,105568,+8801725758667,11/06/2018 8:41:38 AM,0,864541030054991,OPPOA37fw,5.1.1,117,...,Dhaka,Sales Force-SF333,Md Shemul Ahammed,"H-11, R-11, B-E, Chandrima Model Town, Mohamma...",MDMOHAN,210336466,19922690230002423,MD625BF4411R00240,DHAKAMETROHA549670,fraudster
5,0,0,183361,+8801757366150,22/07/2018 7:22:45 AM,10,868373031205182,XiaomiRedmi 5,7.1.2,117,...,Dhaka,Sales Force-MIR003,Md Sajib Hossain,"Vill: Silarchar, Union: Silarchar, Madaripur S...",MDSAZOL,210336466,199338710486019423,MD2A76AY3HWHBH89628,DHAKAMETROHA540808,fraudster
6,0,0,54395,+8801911098151,15/05/2018 9:38:00 AM,0,NaN,NaN,NaN,NaN,...,Dhaka,Nadim BP-Uttara Branch,Ruman Ahmed Khadem,"138/1 Tejgaon, Dhaka",MDAKTERKHAN,210336466,PENDING554872,BRBRAJ170201337,DHAKAMETROLA188421,fraudster
7,0,0,5705,+8801780894414,10/02/2018 12:16:19 PM,0,NaN,NaN,NaN,107,...,Dhaka,SALES FORCE-PTR1776,Mohammad Jahurul Islam,"H-4/1, RD- 10/1,Mirpur-13,Dhaka",MAJEDURRAHMAN,210940816,19937713476000205,BRBVAG170706308,DHAKAMETROHA248411,fraudster
8,0,0,186905,+8801710628626,24/07/2018 3:37:17 AM,1,351739080244336,SymphonyH400,6.0,117,...,Dhaka,Sales Force-KB2400,Md. Amir Hossain,"H-4/I, R-10/1, Mirpur, Dhaka",MAJEDURRAHMAN,210940816,19937713476000205,MB3HA10EE76600625,DHAKAMETROHA273887,fraudster
9,0,0,25850,+8801685184011,18/04/2018 10:37:09 AM,284,864432038491536,HUAWEIHUAWEI LUA-U22,5.1,117,...,Dhaka,NaN,Md. Moinuddin Chisty,"House- 6/2/A, Road- Brorbag, Mirpur-1216, Dhaka",SADDAMHOSSAIN,210998935,19922694813000583,MCDAE1B1VH1J05971,DHAKAMETROHA509341,fraudster


In [30]:
df_fraud

,is_deleted,is_blocked,user_id,phone_no,date_registered,total_rides_as_driver,unique_device_id,device_name,os_version,app_versioncode,...,present_district,sales_person,crm_name,per_block,username,driverlicense,drivernid,vehiclechassis,vehicleplate,acc_type
0,0,0,183299,+8801679350974,22/07/2018 6:48:10 AM,0,NaN,NaN,NaN,NaN,...,Dhaka,Sales Force-UTT022,Rezwana Hafiz,"85731901701,Adom,Paotanahat-5450,Pirgacha Pall...",MDSHARIFULISLAM,21148990,19938517319000300,MD2A14AZ4CWC85046,RANGPURHA135222,fraudster
1,0,0,199528,+8801679350497,30/07/2018 6:04:27 AM,0,NaN,NaN,NaN,117,...,Dhaka,Sales Force-KB1800,Sharmin Akhter Rima,"Shafiqul Vila,Pawtanahat-545 Pirgacha Palli,Ra...",MDSHARIFULISLAM,21148990,19938517319000300,MD2A14AZ4CWC85046,RANGPURHA135222,fraudster
2,0,1,72249,+8801777283248,24/05/2018 1:31:45 PM,6,NaN,XiaomiRedmi Note 5A,7.1.2,117,...,Dhaka,Sales Force-KB100,Md Sajib Hossain,"North Para , Kundapara , Jumarbari , Saghata ,...",MDSULTANMAHAMUD,21151376,5982420423,DHAKAMETROHA347297,DHAKAMETROHA347297,fraudster
3,0,0,197850,+8801923456326,29/07/2018 7:41:31 AM,0,867708031750581,XiaomiRedmi Note 5A,7.1.2,117,...,Dhaka,Fixed Rider,Md Sajib Hossain,"North Para , Kundapara , Jumarbari-5750, Sagha...",MDSULTANMAHAMUD,21151376,5982420423,NaN,NaN,fraudster
4,0,0,105568,+8801725758667,11/06/2018 8:41:38 AM,0,864541030054991,OPPOA37fw,5.1.1,117,...,Dhaka,Sales Force-SF333,Md Shemul Ahammed,"H-11, R-11, B-E, Chandrima Model Town, Mohamma...",MDMOHAN,210336466,19922690230002423,MD625BF4411R00240,DHAKAMETROHA549670,fraudster
5,0,0,183361,+8801757366150,22/07/2018 7:22:45 AM,10,868373031205182,XiaomiRedmi 5,7.1.2,117,...,Dhaka,Sales Force-MIR003,Md Sajib Hossain,"Vill: Silarchar, Union: Silarchar, Madaripur S...",MDSAZOL,210336466,199338710486019423,MD2A76AY3HWHBH89628,DHAKAMETROHA540808,fraudster
6,0,0,54395,+8801911098151,15/05/2018 9:38:00 AM,0,NaN,NaN,NaN,NaN,...,Dhaka,Nadim BP-Uttara Branch,Ruman Ahmed Khadem,"138/1 Tejgaon, Dhaka",MDAKTERKHAN,210336466,PENDING554872,BRBRAJ170201337,DHAKAMETROLA188421,fraudster
7,0,0,5705,+8801780894414,10/02/2018 12:16:19 PM,0,NaN,NaN,NaN,107,...,Dhaka,SALES FORCE-PTR1776,Mohammad Jahurul Islam,"H-4/1, RD- 10/1,Mirpur-13,Dhaka",MAJEDURRAHMAN,210940816,19937713476000205,BRBVAG170706308,DHAKAMETROHA248411,fraudster
8,0,0,186905,+8801710628626,24/07/2018 3:37:17 AM,1,351739080244336,SymphonyH400,6.0,117,...,Dhaka,Sales Force-KB2400,Md. Amir Hossain,"H-4/I, R-10/1, Mirpur, Dhaka",MAJEDURRAHMAN,210940816,19937713476000205,MB3HA10EE76600625,DHAKAMETROHA273887,fraudster
9,0,0,25850,+8801685184011,18/04/2018 10:37:09 AM,284,864432038491536,HUAWEIHUAWEI LUA-U22,5.1,117,...,Dhaka,NaN,Md. Moinuddin Chisty,"House- 6/2/A, Road- Brorbag, Mirpur-1216, Dhaka",SADDAMHOSSAIN,210998935,19922694813000583,MCDAE1B1VH1J05971,DHAKAMETROHA509341,fraudster


In [49]:
dataset_LicenseNumber['drivernid']=pd.to_numeric(dataset_LicenseNumber['drivernid'])

ValueError: Unable to parse string "PENDING554872" at position 6

In [50]:
dataset_LicenseNumber['drivernid'].astype('float64')

ValueError: could not convert string to float: 'PENDING01738848622'

In [117]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
dataset_LicenseNumber=MultiColumnLabelEncoder(columns = ['drivernid','vehiclechassis','vehicleplate','driverlicense','username','crm_name','sales_person','per_block','present_district','profession','date_registered','license_expiry_date','phone_no','device_name','os_version','unique_device_id','app_versioncode']).fit_transform(dataset_LicenseNumber.fillna('0'))

In [55]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = dataset_LicenseNumber.apply(encoder.fit_transform, axis=1)

TypeError: ("'>' not supported between instances of 'float' and 'str'", 'occurred at index 0')

In [57]:
dataset_LicenseNumber.apply(LabelEncoder().fit_transform)

TypeError: ("'>' not supported between instances of 'float' and 'str'", 'occurred at index unique_device_id')

In [63]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
#dataset_LicenseNumber['drivernid'] = le.fit_transform(dataset_LicenseNumber.drivernid.values)
encoder.fit_transform(dataset_LicenseNumber["drivernid"].fillna('0'))
encoder.fit_transform(dataset_LicenseNumber["vehiclechasis"].fillna('0'))
encoder.fit_transform(dataset_LicenseNumber["vehicleplate"].fillna('0'))

array([140, 140, 375, ..., 830, 557, 710], dtype=int64)

In [69]:
dataset_LicenseNumber

,is_deleted,is_blocked,user_id,phone_no,date_registered,total_rides_as_driver,unique_device_id,device_name,os_version,app_versioncode,...,sales_person,crm_name,per_block,username,driverlicense,drivernid,vehiclechassis,vehicleplate,acct_type,fraud
0,0,0,183299,+8801679350974,22/07/2018 6:48:10 AM,0,0,0,0,0,...,Sales Force-UTT022,Rezwana Hafiz,"85731901701,Adom,Paotanahat-5450,Pirgacha Pall...",MDSHARIFULISLAM,21148990,140,361,785,fraudster,1
1,0,0,199528,+8801679350497,30/07/2018 6:04:27 AM,0,0,0,0,117,...,Sales Force-KB1800,Sharmin Akhter Rima,"Shafiqul Vila,Pawtanahat-545 Pirgacha Palli,Ra...",MDSHARIFULISLAM,21148990,140,361,785,fraudster,1
2,0,1,72249,+8801777283248,24/05/2018 1:31:45 PM,6,0,XiaomiRedmi Note 5A,7.1.2,117,...,Sales Force-KB100,Md Sajib Hossain,"North Para , Kundapara , Jumarbari , Saghata ,...",MDSULTANMAHAMUD,21151376,375,200,269,fraudster,1
3,0,0,197850,+8801923456326,29/07/2018 7:41:31 AM,0,867708031750581,XiaomiRedmi Note 5A,7.1.2,117,...,Fixed Rider,Md Sajib Hossain,"North Para , Kundapara , Jumarbari-5750, Sagha...",MDSULTANMAHAMUD,21151376,375,1,0,fraudster,1
4,0,0,105568,+8801725758667,11/06/2018 8:41:38 AM,0,864541030054991,OPPOA37fw,5.1.1,117,...,Sales Force-SF333,Md Shemul Ahammed,"H-11, R-11, B-E, Chandrima Model Town, Mohamma...",MDMOHAN,210336466,122,567,434,fraudster,1
5,0,0,183361,+8801757366150,22/07/2018 7:22:45 AM,10,868373031205182,XiaomiRedmi 5,7.1.2,117,...,Sales Force-MIR003,Md Sajib Hossain,"Vill: Silarchar, Union: Silarchar, Madaripur S...",MDSAZOL,210336466,137,438,389,fraudster,1
6,0,0,54395,+8801911098151,15/05/2018 9:38:00 AM,0,0,0,0,0,...,Nadim BP-Uttara Branch,Ruman Ahmed Khadem,"138/1 Tejgaon, Dhaka",MDAKTERKHAN,210336466,821,128,478,fraudster,1
7,0,0,5705,+8801780894414,10/02/2018 12:16:19 PM,0,0,0,0,107,...,SALES FORCE-PTR1776,Mohammad Jahurul Islam,"H-4/1, RD- 10/1,Mirpur-13,Dhaka",MAJEDURRAHMAN,210940816,139,169,179,fraudster,1
8,0,0,186905,+8801710628626,24/07/2018 3:37:17 AM,1,351739080244336,SymphonyH400,6.0,117,...,Sales Force-KB2400,Md. Amir Hossain,"H-4/I, R-10/1, Mirpur, Dhaka",MAJEDURRAHMAN,210940816,139,247,196,fraudster,1
9,0,0,25850,+8801685184011,18/04/2018 10:37:09 AM,284,864432038491536,HUAWEIHUAWEI LUA-U22,5.1,117,...,0,Md. Moinuddin Chisty,"House- 6/2/A, Road- Brorbag, Mirpur-1216, Dhaka",SADDAMHOSSAIN,210998935,123,308,336,fraudster,1


In [74]:
list(dataset_LicenseNumber)

['is_deleted',
 'is_blocked',
 'user_id',
 'phone_no',
 'date_registered',
 'total_rides_as_driver',
 'unique_device_id',
 'device_name',
 'os_version',
 'app_versioncode',
 'driver_type',
 'license_expiry_date',
 'profession',
 'present_district',
 'sales_person',
 'crm_name',
 'per_block',
 'username',
 'driverlicense',
 'drivernid',
 'vehiclechassis',
 'vehicleplate',
 'acct_type',
 'fraud']

In [126]:
# ---------- top features
top = 10

feats = mdl_fit.feature_importances_
i = np.argsort(feats)[::-1]
cols = np.array(df_fraud.columns.tolist())
top_n = cols[i][:top]
top_feats = feats[i][:top]
tops = zip(top_n,top_feats)
print (tops)

In [127]:
cols

array(['is_deleted', 'is_blocked', 'user_id', 'phone_no',
       'date_registered', 'total_rides_as_driver', 'unique_device_id',
       'device_name', 'os_version', 'app_versioncode', 'driver_type',
       'license_expiry_date', 'profession', 'present_district',
       'sales_person', 'crm_name', 'per_block', 'username',
       'driverlicense', 'drivernid', 'vehiclechassis', 'vehicleplate'], 
      dtype='<U21')

In [128]:
top_n

array(['vehicleplate', 'vehiclechassis', 'is_blocked', 'user_id',
       'phone_no', 'date_registered', 'total_rides_as_driver',
       'unique_device_id', 'device_name', 'os_version'], 
      dtype='<U21')

In [129]:
feats

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [130]:
tops

In [291]:
dataset_onemonth=read_csv("C:/Users/HP/Desktop/test_one_month_csv.csv")

C:\Users\HP\Anaconda3\envs\neuralnets\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [292]:
dataset_onemonth

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
0,Passenger,No,No,Logged In,Free,1493,SIDDIQURRAHMAN,+8801711544904,Male,2018-07-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Passenger,No,No,Logged Out,Busy,4189,MDNASIR,+8801732214292,Male,2018-07-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Passenger,No,No,Logged In,Free,7382,SANIATJAHAN,+8801757547467,Male,2018-07-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Passenger,No,No,Logged In,Free,23946,MDMIZANURRAHAMAN,+8801711186932,Male,2018-08-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Passenger,No,No,Logged Out,Free,67029,MDANISUR,+8801790545844,Male,2018-08-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Passenger,No,No,Logged In,Free,95823,ASIFSHOJOL,+8801701206402,Male,2018-08-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Passenger,No,No,Logged Out,Free,103814,UJJALHOSSAIN,+8801761623871,Male,2018-08-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Passenger,No,No,Logged In,Free,111870,SAMIULISLAM,+8801992822272,Male,2018-08-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Passenger,No,No,Logged In,Free,115523,JAHO,+8801819505606,Male,2018-08-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Passenger,No,No,Logged In,Busy,118389,ALAMGIRHOSSAUN,+8801674013545,Male,2018-08-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [293]:
dataset_result=dataset_onemonth.sort_values(["driverlicense","username"], inplace=True, ascending=True)

In [294]:
dataset_result

In [149]:
dataset_onemonth['driverlicense'].value_counts()

211598607             2
211473513             2
DK0332528L00009       2
DK0410943CL0007       2
NR0019001CL0002       2
211683536             2
DK0517890L00006       2
CG0046772CL0009       2
6211616766            1
DK0671983CL0005       1
211631681             1
6211599400            1
CG0124601C00006       1
DK0735622CL0001       1
6211560838            1
5011739576            1
21148990              1
DK0073401CL0001       1
5011464319            1
CG0021454L00003       1
211678303             1
PENDING5464           1
DK0739118CL0003       1
DK0745589CL0003       1
DK0641871CL0001       1
211565896             1
411650363             1
211623129             1
CG0105160CL0006       1
DK0128244CL0009       1
                     ..
DK0480214C00008       1
211576241             1
410325390             1
210635606             1
211665924             1
5011527803            1
211657114             1
DK0372320CL0009       1
4511669392            1
DK0029660C00000       1
4211823468      

In [151]:
dataset_onemonth.dropna()

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
37838,Driver,No,No,Logged In,Free,195826,CHOWDHURYSHAHRIARSWARUP,+8801633618990,Male,2018-07-28,...,MOTO,COMMISSION,2111657883,06-03-2024,DHAKAMETROHA552847,PS0JC7190JH208827,--,Sales Force- MIR906,Rezwana Hafiz,DHAKA


In [ ]:
dataset_driver=DataFrame()
i=0
for row in range(len(dataset_onemonth)):
    if dataset_onemonth['type'][row]=='Driver':
        dataset_driver.loc[i]=dataset_onemonth

In [175]:
dataset_twomonth=read_csv("C:/Users/HP/Desktop/driver.csv")

In [176]:
dataset_twomonth

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
0,Driver,No,No,Logged In,Free,132459,NAYANDAS,8.80182E+12,Male,6/28/2018,...,MOTO,COMMISSION,CG0035777C00007,6/1/2023,CHATTAMETROHA139996,MD2AJ4AZ5CWD92879,NaN,Sales Force-CTG_NELOY,Md Enamul Islam,CHITTAGONG
1,Driver,No,No,Logged In,Free,132464,SYEDSHAMSULHAQUE,8.80172E+12,Male,6/28/2018,...,MOTO,COMMISSION,1011548825,19-09-2022,SYLHETHA131356,LZLK1325763C64692,--,Ariful Islam Linkon,Jahurul Islam,SYLHET
2,Driver,No,No,Logged In,Free,132485,MOHAMMEDYAHEYA,8.80183E+12,Male,6/28/2018,...,MOTO,COMMISSION,CG0119218C00000,7/5/2026,CHATTAMETROHA144323,LXSPCGLY1A4067418,NaN,Sales Force-CTG_FAHIM,Rehana Akter Runa,CHITTAGONG
3,Driver,No,No,Logged In,Free,132491,MDSALIMBAPARI,8.80191E+12,Male,6/28/2018,...,CNG,COMMISSION,DK0513265TM0008,13-08-2021,DHAKADWA140453,MD2AAAFZZTWA01359,--,CNG-BP-Md Enamul Haque,Md Habibur Rahman,DHAKA
4,Driver,No,No,Logged In,Free,132509,ABULKALAMAZAD,8.80188E+12,Male,6/28/2018,...,MOTO,COMMISSION,CG0151705C00003,14-11-2027,CHATTAMETROHA118331,LATPCDLJ761110258,NaN,Sales Force-CTG_FAHIM,Md Shemul Ahammed,CHITTAGONG
5,Driver,No,No,Logged In,Free,132518,SHEIKHMDSOWKATALI,8.80171E+12,Male,6/28/2018,...,MOTO,COMMISSION,DK0490480CL0005,12/9/2025,DHAKAMETROLA187167,BRBRAJ170201518,NaN,Sales Force-SF222,Md Enamul Islam,DHAKA
6,Driver,No,No,Logged In,Free,132545,MDARIFHOSSAIN,8.80192E+12,Male,6/28/2018,...,MOTO,COMMISSION,CG0148368C0000,9/10/2027,CHATTAMETROHA158129,PS1JAS090HJE01141,NaN,Sales Force-CTG_TRISHAN,Rezwana Hafiz,CHITTAGONG
7,Driver,No,No,Logged Out,Free,132585,MDTOYBURRAHMAN,8.80186E+12,Male,6/28/2018,...,CNG,COMMISSION,DK0137844T00001,9/3/2022,DHAKAMETROTAW131693,AAMBKA00936,NaN,CNG-BP-Md Liton Gazi,MD. Amir Hossain,DHAKA
8,Driver,No,No,Logged In,Free,132588,MDNURUDDIN,8.80182E+12,Male,6/28/2018,...,MOTO,COMMISSION,DK0226182CL0007,21-07-2023,CHATTAMETROHA153472,MD2A15BZ0GWK89215,NaN,Sales Force-CTG_SHUVO,Rehana Akter Runa,CHITTAGONG
9,Driver,No,No,Logged In,Free,132631,MDJEWEL,8.80177E+12,Male,6/28/2018,...,MOTO,COMMISSION,DK0031471CL0008,16-01-2022,DHAKAMETROLA179941,MD2DHDH22RCE69893,NaN,Sales Force-SF222,Md Enamul Islam,DHAKA


In [162]:
dataset_twomonth.sort_values(["driverlicense"], inplace=True, ascending=True)

In [174]:
dataset_twomonth['driverlicense'].value_counts()

211573138                  2
211394003                  2
211598607                  2
MB0000167CL0006            2
DK0692866L00009            2
DK0576119L00009            2
211385501                  2
DK0675697CL0002            2
NR0019001CL0002            2
DK0517890L00006            2
SR0003736C00008            2
411601103                  2
CG0034132L00000            2
DK0228644L00004            2
GP0020753TL0006            2
DK0473906L00009            2
211577757                  2
CG2301209L00012            2
DK0445241CL0000            2
411399639                  2
211473513                  2
DK0410943CL0007            2
MK0007620CL0006            2
211599090                  2
411627504                  2
CG0155272L00003            2
CG0008585L00001            2
DK0108130L00009            2
DK0138853T00002            2
DK0461411L00007            2
                          ..
DK0381479CL0006            1
4511561433                 1
211654702                  1
411617517     

In [172]:
dataset_twomonth['sort'] = dataset_twomonth['driverlicense'].str.extract('(\d+)', expand=False).astype('float64')
dataset_twomonth.sort_values('sort',inplace=True, ascending=True)
dataset_twomonth = dataset_twomonth.drop('sort', axis=1)

In [177]:
dataset_twomonth[dataset_twomonth.duplicated(['driverlicense'], keep=False)]

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
26,Driver,No,Yes,Logged Out,Free,132892,MDJAHANGIRALAM,8.80199E+12,Male,6/28/2018,...,CNG,COMMISSION,DK0108130L00009,7/8/2019,DHAKAMETROTAW130862,AFFBJL13973,--,CNG-BP- Arobi Akter,Rezwana Hafiz,DHAKA
39,Driver,No,Yes,Logged Out,Free,133155,MDSUMON,8.80171E+12,Male,6/28/2018,...,CNG,COMMISSION,GP0020753TL0006,7/7/2019,DHAKADWA111110,MD2AAAFZZUWE14753,--,CNG-BP-Md Sohel Mia,Md Habibur Rahman,DHAKA
54,Driver,No,No,Logged Out,Free,133531,PRAKASHDEY,8.80182E+12,Male,6/28/2018,...,MOTO,COMMISSION,CG0008585L00001,17-06-2022,CHATTAMETROHA115992,0SF09C28905,NaN,Sales Force-CTG_TRISHAN,Md Shemul Ahammed,CHITTAGONG
263,Driver,Yes,Yes,Logged Out,Free,138828,HASANJAYED,kHbzDELETED+8801672674080,Male,7/1/2018,...,MOTO,COMMISSION,DK0445241CL0000,29-03-2026,DHAKAMETROLA173984,MBLKC12ECB6A05077,NaN,Md. Ariful Islam Linkon,Rezwana Hafiz,DHAKA
266,Driver,Yes,Yes,Logged Out,Free,138889,ELIASELIAS,0PjyDELETED+8801720369975,Male,7/1/2018,...,NaN,COMMISSION,DK0675697CL0002,16-01-2028,NaN,NaN,NaN,Sales Force-KB100,Md Enamul Islam,DHAKA
321,Driver,No,Yes,Logged Out,Free,139674,MDNIROB,8.80172E+12,Male,7/1/2018,...,CNG,COMMISSION,DK0196336L00904,26-02-2020,DHAKADWA110945,MD2AAAFZZUWD11053,NaN,CNG-BP-Md Shohel,Ruman Ahmed Khadem,DHAKA
361,Driver,No,No,Logged In,Free,140192,MDSHAMIMAHMED,8.80194E+12,Male,7/1/2018,...,MOTO,COMMISSION,DK0638913CL0006,24-10-2027,DHAKAMETROHA461564,LZL12P224DHG58236,NaN,KB Branch,Rezwana Hafiz,DHAKA
390,Driver,No,No,Logged Out,Free,141239,MREMRAN,8.80185E+12,Male,7/2/2018,...,CNG,COMMISSION,DK0338281TL0004,10/12/2019,DHAKAMETROTAW142291,AFFBKA00132,--,CNG-BP-Shahadat Hossain,Dibakar Chandra Das,DHAKA
423,Driver,No,Yes,Logged Out,Free,141599,MEHEDIHASSANSHOHEL,8.8019E+12,Male,7/2/2018,...,MOTO,COMMISSION,411601103,12/12/2018,CHATTAMETROHA160373,PS1JAS090HJJ00026,NaN,Sales Force-CTG_PRATIK 82454,Oindrila Mahzabeen Chowdhury,CHITTAGONG
537,Driver,No,No,Logged Out,Free,143630,MDBADRUDDOZA,8.80186E+12,Male,7/3/2018,...,MOTO,COMMISSION,CG0034132L00000,12/12/2022,CHATTAMETROLA144571,MD624HC18H2G74473,--,Md Ariful Islam Linkon,Oindrila Mahzabeen Chowdhury,CHITTAGONG


In [195]:
duplicate_driverlicense=pd.concat(g for _, g in dataset_twomonth.groupby("driverlicense") if len(g) > 1)

In [193]:
duplicate_nid=pd.concat(g for _, g in dataset_twomonth.groupby("nid") if len(g) > 1)
duplicate_chassis=pd.concat(g for _, g in dataset_twomonth.groupby("vehiclechassis") if len(g) > 1)
duplicate_numberplate=pd.concat(g for _, g in dataset_twomonth.groupby("vehicleplate") if len(g) > 1)

In [207]:
# DF TO EXCEL
from pandas import ExcelWriter

writer = ExcelWriter('duplicate_NID.xlsx')
duplicate_nid.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_DriverLicense.xlsx')
duplicate_driverlicense.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_VehicleChassis.xlsx')
duplicate_chassis.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_VehicleNumberPlate.xlsx')
duplicate_numberplate.to_excel(writer,'Sheet5')
writer.save()

# DF TO CSV
#duplicate_nid.to_csv('duplicate_NID.csv', sep=',')

In [194]:
duplicate_numberplate['vehiclechassis']

3979      PENDING6354353135
4772       PENDING443545353
652            NCP580028101
3744           NCP580028101
625           NZE1213313303
1435          NZE1213313303
1041          NZE1416118846
3385          NZE1416118846
1920      MBLHA10A6EGB09764
7136      MBLHA10A6EGB09764
423       PS1JAS090HJJ00026
1762      PS1JAS090HJJ00026
4044      PS1HAR146HJJ00154
5544       PS1HAR14HJJ00154
1749      MD2A85CY3HCE07278
3891      MD2A85CY3HCE07278
284       MD2AAAFZZTWG23767
5596      MD2AAAFZZTWG23767
314       MD2AAAFZZTWJ29240
4828      MD2AAAFZZTWJ29240
1684      MD2AAAFZZUWA02453
2479      MD2AAAFZZUWA02453
3368      MD2AAAFZZUWA02453
46        MD2AAAFZZUWA03224
6907      MD2AAAFZZUWA03224
321       MD2AAAFZZUWD11053
1876      MD2AAAFZZUWD11053
39        MD2AAAFZZUWE14753
3246      MD2AAAFZZUWE14753
763       MD2AAAFZZUWE14929
               ...         
357             AFFBKD07040
1547            AFFBKD07040
1268      MD2AAAFZZTWF19119
1284      MD2AAAFZZTWF19119
1208      MD2A27AZ0F

In [210]:
duplicate_numberplate['contact']

3979                  8.80185E+12
4772                  8.80168E+12
652                   8.80183E+12
3744                    8.802E+12
625                   8.80164E+12
1435                  8.80172E+12
1041                  8.80182E+12
3385                  8.80186E+12
1920    eY0wDELETED+8801628245803
7136                  8.80172E+12
423                    8.8019E+12
1762                  8.80181E+12
4044                  8.80185E+12
5544                   8.8018E+12
1749                  8.80168E+12
3891                  8.80181E+12
284                   8.80185E+12
5596                  8.80164E+12
314                   8.80191E+12
4828                  8.80192E+12
1684                  8.80178E+12
2479                  8.80198E+12
3368                  8.80191E+12
46                    8.80173E+12
6907                  8.80194E+12
321                   8.80172E+12
1876                  8.80172E+12
39                    8.80171E+12
3246                  8.80184E+12
763           

In [211]:
#duplicate_numberplate.dtypes
duplicate_numberplate.astype(object).convert_objects()

C:\Users\HP\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
3979,Driver,No,No,Logged In,Free,193010,SAMSULISLAMFAISAL,8.80185E+12,Male,7/27/2018,...,MOTO,COMMISSION,411591392,5/3/2020,A.F.R.HA359586,PENDING6354353135,NaN,Sales Force-CTG_FAHIM,Ruman Ahmed Khadem,CHITTAGONG
4772,Driver,No,No,Logged In,Free,204815,MDNAZIMUDDIN,8.80168E+12,Male,8/2/2018,...,MOTO,COMMISSION,411625316,5/3/2020,A.F.R.HA359586,PENDING443545353,NaN,Sales Force-CTG_SOJAL,Ruman Ahmed Khadem,CHITTAGONG
652,Driver,No,Yes,Logged Out,Free,144859,MDKAMRULHASSAN,8.80183E+12,Male,7/3/2018,...,GARI,COMMISSION,CG0155272L00003,27-12-2022,CHATTAMETROGA117712,NCP580028101,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
3744,Driver,No,No,Logged In,Free,189931,MDKAMRULHASSAN,8.802E+12,Male,7/25/2018,...,GARI,COMMISSION,CG0155272L00003,27-12-2022,CHATTAMETROGA117712,NCP580028101,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
625,Driver,No,Yes,Logged Out,Free,144473,MOHAMMEDIMRANAHMED,8.80164E+12,Male,7/3/2018,...,GARI,COMMISSION,CG2301209L00012,20-12-2019,CHATTAMETROGA124962,NZE1213313303,NaN,Sales Force-CTG_AHASAN,Rezwana Hafiz,CHITTAGONG
1435,Driver,No,No,Logged In,Free,157608,MOHAMMEDIMRANAHMED,8.80172E+12,Male,7/9/2018,...,GARI,COMMISSION,CG2301209L00012,20-12-2019,CHATTAMETROGA124962,NZE1213313303,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
1041,Driver,No,No,Logged In,Free,153136,MDIDRIS,8.80182E+12,Male,7/7/2018,...,GARI,COMMISSION,DK0228644L00004,29-11-2018,CHATTAMETROGA129297,NZE1416118846,NaN,Sales Force-CTG_FAHIM,Dibakar Chandra Das,CHITTAGONG
3385,Driver,No,No,Logged In,Free,185659,MDIDRIS,8.80186E+12,Male,7/23/2018,...,GARI,COMMISSION,DK0228644L00004,29-11-2018,CHATTAMETROGA129297,NZE1416118846,NaN,Sales Force-CTG_FAHIM,Dibakar Chandra Das,CHITTAGONG
1920,Driver,Yes,Yes,Logged Out,Free,165483,MDRASHEDSAROWARCHY,eY0wDELETED+8801628245803,Male,7/12/2018,...,MOTO,COMMISSION,CG0055539C,28-12-2018,CHATTAMETROHA144546,MBLHA10A6EGB09764,NaN,Sales Force-CTG_SHUVO,Dibakar Chandra Das,CHITTAGONG
7136,Driver,No,No,Logged In,Free,239746,MDRASHEDSAROWARCHY,8.80172E+12,Male,8/20/2018,...,MOTO,COMMISSION,CG0055539C,18-08-2021,CHATTAMETROHA144546,MBLHA10A6EGB09764,NaN,Sales Force-CTG_SHUVO,Oindrila Mahzabeen Chowdhury,CHITTAGONG


In [215]:
#from io import StringIO
#read_csv(StringIO(dataset_twomonth),sep='\s+')

In [217]:
dataset_driver_excel=read_csv("C:/Users/HP/Desktop/driver_excel.csv")
dataset_driver_excel

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
0,Driver,No,No,Logged In,Free,132459,NAYANDAS,+8801815913550,Male,6/28/2018,...,MOTO,COMMISSION,CG0035777C00007,06-01-2023,CHATTAMETROHA139996,MD2AJ4AZ5CWD92879,NaN,Sales Force-CTG_NELOY,Md Enamul Islam,CHITTAGONG
1,Driver,No,No,Logged In,Free,132464,SYEDSHAMSULHAQUE,+8801715196396,Male,6/28/2018,...,MOTO,COMMISSION,1011548825,19-09-2022,SYLHETHA131356,LZLK1325763C64692,--,Ariful Islam Linkon,Jahurul Islam,SYLHET
2,Driver,No,No,Logged In,Free,132485,MOHAMMEDYAHEYA,+8801831577709,Male,6/28/2018,...,MOTO,COMMISSION,CG0119218C00000,07-05-2026,CHATTAMETROHA144323,LXSPCGLY1A4067418,NaN,Sales Force-CTG_FAHIM,Rehana Akter Runa,CHITTAGONG
3,Driver,No,No,Logged In,Free,132491,MDSALIMBAPARI,+8801905773522,Male,6/28/2018,...,CNG,COMMISSION,DK0513265TM0008,13-08-2021,DHAKADWA140453,MD2AAAFZZTWA01359,--,CNG-BP-Md Enamul Haque,Md Habibur Rahman,DHAKA
4,Driver,No,No,Logged In,Free,132509,ABULKALAMAZAD,+8801876408800,Male,6/28/2018,...,MOTO,COMMISSION,CG0151705C00003,14-11-2027,CHATTAMETROHA118331,LATPCDLJ761110258,NaN,Sales Force-CTG_FAHIM,Md Shemul Ahammed,CHITTAGONG
5,Driver,No,No,Logged In,Free,132518,SHEIKHMDSOWKATALI,+8801712727028,Male,6/28/2018,...,MOTO,COMMISSION,DK0490480CL0005,12-09-2025,DHAKAMETROLA187167,BRBRAJ170201518,NaN,Sales Force-SF222,Md Enamul Islam,DHAKA
6,Driver,No,No,Logged In,Free,132545,MDARIFHOSSAIN,+8801917905675,Male,6/28/2018,...,MOTO,COMMISSION,CG0148368C0000,09-10-2027,CHATTAMETROHA158129,PS1JAS090HJE01141,NaN,Sales Force-CTG_TRISHAN,Rezwana Hafiz,CHITTAGONG
7,Driver,No,No,Logged Out,Free,132585,MDTOYBURRAHMAN,+8801861041269,Male,6/28/2018,...,CNG,COMMISSION,DK0137844T00001,09-03-2022,DHAKAMETROTAW131693,AAMBKA00936,NaN,CNG-BP-Md Liton Gazi,MD. Amir Hossain,DHAKA
8,Driver,No,No,Logged In,Free,132588,MDNURUDDIN,+8801818037812,Male,6/28/2018,...,MOTO,COMMISSION,DK0226182CL0007,21-07-2023,CHATTAMETROHA153472,MD2A15BZ0GWK89215,NaN,Sales Force-CTG_SHUVO,Rehana Akter Runa,CHITTAGONG
9,Driver,No,No,Logged In,Free,132631,MDJEWEL,+8801772976011,Male,6/28/2018,...,MOTO,COMMISSION,DK0031471CL0008,16-01-2022,DHAKAMETROLA179941,MD2DHDH22RCE69893,NaN,Sales Force-SF222,Md Enamul Islam,DHAKA


In [218]:
duplicate_driverlicense=pd.concat(g for _, g in dataset_driver_excel.groupby("driverlicense") if len(g) > 1)
duplicate_nid=pd.concat(g for _, g in dataset_driver_excel.groupby("nid") if len(g) > 1)
duplicate_chassis=pd.concat(g for _, g in dataset_driver_excel.groupby("vehiclechassis") if len(g) > 1)
duplicate_numberplate=pd.concat(g for _, g in dataset_driver_excel.groupby("vehicleplate") if len(g) > 1)

In [229]:
#duplicate_driverlicense
#duplicate_nid
#duplicate_chassis
duplicate_numberplate

,type,deleted,blocked,onlinestatus,status,id,username,contact,gender,regdate,...,vehicletype,drivertype,driverlicense,licensedate,vehicleplate,vehiclechassis,profession,salesperson,crm,city
3979,Driver,No,No,Logged In,Free,193010,SAMSULISLAMFAISAL,+8801846323199,Male,7/27/2018,...,MOTO,COMMISSION,411591392,05-03-2020,A.F.R.HA359586,PENDING6354353135,NaN,Sales Force-CTG_FAHIM,Ruman Ahmed Khadem,CHITTAGONG
4772,Driver,No,No,Logged In,Free,204815,MDNAZIMUDDIN,+8801676305288,Male,8/2/2018,...,MOTO,COMMISSION,411625316,05-03-2020,A.F.R.HA359586,PENDING443545353,NaN,Sales Force-CTG_SOJAL,Ruman Ahmed Khadem,CHITTAGONG
652,Driver,No,Yes,Logged Out,Free,144859,MDKAMRULHASSAN,+8801834827399,Male,7/3/2018,...,GARI,COMMISSION,CG0155272L00003,27-12-2022,CHATTAMETROGA117712,NCP580028101,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
3744,Driver,No,No,Logged In,Free,189931,MDKAMRULHASSAN,+8801995809295,Male,7/25/2018,...,GARI,COMMISSION,CG0155272L00003,27-12-2022,CHATTAMETROGA117712,NCP580028101,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
625,Driver,No,Yes,Logged Out,Free,144473,MOHAMMEDIMRANAHMED,+8801643522166,Male,7/3/2018,...,GARI,COMMISSION,CG2301209L00012,20-12-2019,CHATTAMETROGA124962,NZE1213313303,NaN,Sales Force-CTG_AHASAN,Rezwana Hafiz,CHITTAGONG
1435,Driver,No,No,Logged In,Free,157608,MOHAMMEDIMRANAHMED,+8801715549303,Male,7/9/2018,...,GARI,COMMISSION,CG2301209L00012,20-12-2019,CHATTAMETROGA124962,NZE1213313303,NaN,Sales Force-CTG_AHASAN,Dibakar Chandra Das,CHITTAGONG
1041,Driver,No,No,Logged In,Free,153136,MDIDRIS,+8801816169079,Male,7/7/2018,...,GARI,COMMISSION,DK0228644L00004,29-11-2018,CHATTAMETROGA129297,NZE1416118846,NaN,Sales Force-CTG_FAHIM,Dibakar Chandra Das,CHITTAGONG
3385,Driver,No,No,Logged In,Free,185659,MDIDRIS,+8801861690799,Male,7/23/2018,...,GARI,COMMISSION,DK0228644L00004,29-11-2018,CHATTAMETROGA129297,NZE1416118846,NaN,Sales Force-CTG_FAHIM,Dibakar Chandra Das,CHITTAGONG
1920,Driver,Yes,Yes,Logged Out,Free,165483,MDRASHEDSAROWARCHY,eY0wDELETED+8801628245803,Male,7/12/2018,...,MOTO,COMMISSION,CG0055539C,28-12-2018,CHATTAMETROHA144546,MBLHA10A6EGB09764,NaN,Sales Force-CTG_SHUVO,Dibakar Chandra Das,CHITTAGONG
7136,Driver,No,No,Logged In,Free,239746,MDRASHEDSAROWARCHY,+8801715125686,Male,8/20/2018,...,MOTO,COMMISSION,CG0055539C,18-08-2021,CHATTAMETROHA144546,MBLHA10A6EGB09764,NaN,Sales Force-CTG_SHUVO,Oindrila Mahzabeen Chowdhury,CHITTAGONG


In [236]:
# DF TO EXCEL
from pandas import ExcelWriter

writer = ExcelWriter('duplicate_NID.xlsx')
duplicate_nid.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_DriverLicense.xlsx')
duplicate_driverlicense.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_VehicleChassis.xlsx')
duplicate_chassis.to_excel(writer,'Sheet5')
writer.save()

writer = ExcelWriter('duplicate_VehicleNumberPlate.xlsx')
duplicate_numberplate.to_excel(writer,'Sheet5')
writer.save()

# DF TO CSV
#duplicate_nid.to_csv('duplicate_NID.csv', sep=',')

In [182]:
list(dataset_twomonth)

['type',
 'deleted',
 'blocked',
 'onlinestatus',
 'status',
 'id',
 'username',
 'contact',
 'gender',
 'regdate',
 'regtime',
 'lastlogin',
 'lastupdate',
 'totalrides',
 'regdevice',
 'currentdevice',
 'devicetype',
 'devicename',
 'osversion',
 'versioncode',
 'currentlatlong',
 'nid',
 'currentregion',
 'available',
 'vehicletype',
 'drivertype',
 'driverlicense',
 'licensedate',
 'vehicleplate',
 'vehiclechassis',
 'profession',
 'salesperson',
 'crm',
 'city']

In [2]:
from sqlalchemy import create_engine
import pandas as pd
from pandas import DataFrame
engine_from = create_engine('mysql+pymysql://root:??&&OBHAI>>IS*({StiLl}<<AliVe)]@52.76.132.145:3306/obhai_live')

In [321]:
sql = "SELECT * FROM tb_engagements WHERE tb_engagements.request_made_on Between DATE_ADD(NOW() , INTERVAL -1 MONTH) and NOW() AND tb_engagements.status=3"
dataset_one_week = pd.read_sql_query(sql, engine_from)

In [5]:
sql = "SELECT user_id,access_token FROM tb_users WHERE tb_users.date_registered Between DATE_ADD(NOW() , INTERVAL -1 WEEK) and NOW()"
dataset_one_week_access_token = pd.read_sql_query(sql, engine_from)

In [6]:
duplicate_access_token=pd.concat(g for _, g in dataset_one_week_access_token.groupby("access_token") if len(g) > 1)

In [7]:
duplicate_access_token

,user_id,access_token
24,258608,
1668,260252,
2017,260602,
2046,260631,
2077,260662,
3275,261866,
3401,261993,
3753,262344,
7334,265917,
7834,266419,


In [8]:
# DF TO EXCEL
from pandas import ExcelWriter

writer = ExcelWriter('Duplicate_Access_Token.xlsx')
duplicate_access_token.to_excel(writer,'Sheet5')
writer.save()


In [317]:
dataset_result_one_week=dataset_one_week.sort_values(['pickup_time', 'drop_time'], ascending=[True, True], inplace=False)
dataset_one_week['drop_time']

5       2018-08-01 10:46:35
9       2018-08-01 10:59:19
12      2018-08-01 11:10:23
13      2018-08-01 11:34:50
15      2018-08-01 10:59:32
17      2018-08-01 10:52:49
6       2018-08-01 11:00:39
18      2018-08-01 10:52:02
20      2018-08-01 10:48:51
19      2018-08-01 11:08:54
10      2018-08-01 11:06:20
0       2018-08-01 11:02:23
21      2018-08-01 10:59:58
22      2018-08-01 11:07:07
23      2018-08-01 10:51:58
1       2018-08-01 12:04:23
24      2018-08-01 11:23:52
26      2018-08-01 10:49:25
14      2018-08-01 10:56:54
32      2018-08-01 10:55:54
33      2018-08-01 11:24:51
34      2018-08-01 11:15:32
36      2018-08-01 10:51:34
27      2018-08-01 10:54:50
35      2018-08-01 10:56:26
42      2018-08-01 10:56:07
16      2018-08-01 11:03:01
43      2018-08-01 11:45:50
30      2018-08-01 10:59:59
48      2018-08-01 10:56:41
                ...        
60855   2018-09-01 10:03:26
60856   2018-09-01 10:08:52
60858   2018-09-01 10:06:27
60857   2018-09-01 10:04:53
60849   2018-09-01 1

In [309]:
dataset_list=dataset_one_week.groupby('driver_id')['user_id'].apply(list).reset_index()

In [310]:
dataset_list.index

RangeIndex(start=0, stop=3671, step=1)

In [320]:
dataset_list

,driver_id,user_id
0,3,"[0, 31, 624, 5296, 12869, 15429, 21845, 32512,..."
1,5,"[0, 23, 45, 70, 162, 2852, 4814, 7856, 10273, ..."
2,10,"[3075, 11890, 14281, 113313, 149814, 174241, 1..."
3,13,"[0, 12034, 18376, 37444, 141751, 157681, 17752..."
4,14,"[0, 23, 122, 227, 271, 1247, 2852, 3803, 4269,..."
5,15,"[0, 46, 523, 526, 10997, 11030, 14778, 16450, ..."
6,16,"[38, 227, 10592, 13062, 15869, 20199, 59511, 6..."
7,18,"[0, 23, 148, 162, 268, 520, 4121, 6538, 7832, ..."
8,257,"[0, 1068, 3558, 6968, 8226, 9292, 10592, 10997..."
9,258,"[0, 23, 70, 285, 813, 1374, 3532, 3579, 4125, ..."


In [280]:
dataset_list['user_id'][0]=sorted(set(dataset_list['user_id'][0]))

C:\Users\HP\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [286]:
dataset_list['user_id'][0][1]

31

In [319]:
for row in range(len(dataset_list)):
    dataset_list['user_id'][row]=sorted(set(dataset_list['user_id'][row]))

C:\Users\HP\Anaconda3\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [323]:
import json, urllib
from urllib import urlencode
import googlemaps
start = "Bridgewater, Sa, Australia"
finish = "Stirling, SA, Australia"

url = 'http://maps.googleapis.com/maps/api/directions/json?%s' % urlencode((
            ('origin', start),
            ('destination', finish)
 ))
ur = urllib.urlopen(url)
result = json.load(ur)

for i in range (0, len (result['routes'][0]['legs'][0]['steps'])):
    j = result['routes'][0]['legs'][0]['steps'][i]['html_instructions'] 
    print (j)

ImportError: cannot import name 'urlencode'

In [20]:
#example obj data containing lat and lng points
#stop location - the radii end point
endpoint_lat = 44.9631
endpoint_lng = -93.2492

#bus location from the southeast - the circle center
startpoint_lat = 44.95517
startpoint_lng = -93.2427

import math
from math import atan2
def function_vehicleBearing(x1,y1,x2,y2):
    endpoint_lat = x1
    endpoint_lng = y1
    startpoint_lat = x2
    startpoint_lng = y2

    #radians = function_getAtan2((y1 - y2), (x1 - x2))

    def function_getAtan2(y, x):
        return atan2(y, x)
    
    radians = function_getAtan2((y1 - y2), (x1 - x2))

    compassReading = radians * (180 / math.pi)

    coordNames = ["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"]
    coordIndex = round(compassReading / 45)
    if (coordIndex < 0):
        coordIndex = coordIndex + 8
    

    return coordNames[coordIndex] # returns the coordinate value


In [21]:
function_vehicleBearing(44.9631,-93.2492,44.95517,-93.2427)

'NW'

In [ ]:
#dataset_twomonth.loc[dataset_twomonth['count'].idxmax()]

In [ ]:
#dataset_twomonth_result=dataset_twomonth.groupby(['drvierlis','id','username']).id.agg('count').to_frame('count')